# Operationalizing Ethics in AI Development & Deployment — Recruitment Context

1. Dataset Creation


In [2]:
import pandas as pd
import numpy as np

np.random.seed(123)

# Synthetic Recruitment Dataset
n = 1200
genders = ["Male", "Female", "Other"]
ethnicities = ["Group_A", "Group_B", "Group_C"]
education_levels = ["High School", "Bachelors", "Masters", "PhD"]

data = pd.DataFrame({
    "Applicant_ID": range(1, n+1),
    "Age": np.random.randint(20, 60, n),
    "Gender": np.random.choice(genders, n, p=[0.5, 0.45, 0.05]),
    "Ethnicity": np.random.choice(ethnicities, n, p=[0.4, 0.35, 0.25]),
    "Education": np.random.choice(education_levels, n, p=[0.3, 0.4, 0.2, 0.1]),
    "Years_Experience": np.random.randint(0, 20, n),
    "Skill_Score": np.random.randint(40, 100, n)  # Out of 100
})

# Introduce a bias pattern
data["Shortlisted"] = np.where(
    (data["Education"].isin(["Masters", "PhD"])) & (data["Skill_Score"] > 70),
    np.random.choice([1, 0], n, p=[0.75, 0.25]),
    np.random.choice([1, 0], n, p=[0.35, 0.65])
)

# Slight advantage for Ethnicity = Group_A
mask_group_a = data["Ethnicity"] == "Group_A"
data.loc[mask_group_a, "Shortlisted"] = np.random.choice([1, 0], mask_group_a.sum(), p=[0.55, 0.45])

data.head()

,Applicant_ID,Age,Gender,Ethnicity,Education,Years_Experience,Skill_Score,Shortlisted
0,1,22,Male,Group_B,Bachelors,12,87,0
1,2,48,Other,Group_B,Bachelors,12,89,1
2,3,54,Male,Group_B,Bachelors,19,45,0
3,4,58,Male,Group_C,Masters,8,41,0
4,5,37,Male,Group_C,Bachelors,16,48,0


Part A — Operationalizing Ethics in Development


Task 1 — Bias Detection in Development


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Prepare data
X = pd.get_dummies(data.drop(columns=["Applicant_ID", "Shortlisted"]), drop_first=True)
y = data["Shortlisted"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train baseline model
model = RandomForestClassifier(n_estimators=50, random_state=42)
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)
print("Baseline Accuracy:", round(accuracy_score(y_test, y_pred), 3))

# Merge back with sensitive attributes for bias check
df_test = X_test.copy()
df_test["y_true"] = y_test
df_test["y_pred"] = y_pred
df_test["Gender"] = data.loc[y_test.index, "Gender"]
df_test["Ethnicity"] = data.loc[y_test.index, "Ethnicity"]

# Bias by Gender
print("\nPositive prediction rate by Gender:")
print(df_test.groupby("Gender")["y_pred"].mean())

# Bias by Ethnicity
print("\nPositive prediction rate by Ethnicity:")
print(df_test.groupby("Ethnicity")["y_pred"].mean())

Baseline Accuracy: 0.544

Positive prediction rate by Gender:
Gender
Female    0.41875
Male      0.38587
Other     0.68750
Name: y_pred, dtype: float64

Positive prediction rate by Ethnicity:
Ethnicity
Group_A    0.582090
Group_B    0.298507
Group_C    0.336957
Name: y_pred, dtype: float64


Task 2 — Fairness Testing in Development


In [4]:
def fairness_check(df, sensitive_attr, threshold=0.1):
    rates = df.groupby(sensitive_attr)["y_pred"].mean()
    disparity = rates.max() - rates.min()
    print(f"{sensitive_attr} Disparity: {round(disparity, 3)}")
    return disparity <= threshold

print("\nFairness Checks:")
gender_fair = fairness_check(df_test, "Gender")
ethnic_fair = fairness_check(df_test, "Ethnicity")

if not (gender_fair and ethnic_fair):
    print("❌ Fairness check FAILED — Mitigation required.")
else:
    print("✅ Fairness check PASSED.")


Fairness Checks:
Gender Disparity: 0.302
Ethnicity Disparity: 0.284
❌ Fairness check FAILED — Mitigation required.


Task 3 — Feature Role & Ethics Review


In [5]:
importances = pd.Series(model.feature_importances_, index=X.columns).sort_values(ascending=False)
print("\nTop Features (Importance):")
print(importances.head(10))


Top Features (Importance):
Skill_Score              0.298657
Age                      0.272228
Years_Experience         0.226355
Gender_Male              0.039310
Education_High School    0.034004
Education_Masters        0.033359
Ethnicity_Group_B        0.031402
Ethnicity_Group_C        0.030982
Education_PhD            0.020676
Gender_Other             0.013026
dtype: float64


Task 4 — Ethics-Aware Agile User Stories
Example user stories:

US1: As a recruiter, I want the AI model to evaluate candidates without bias based on gender so that all applicants are treated fairly.
US2: As a compliance officer, I want automated fairness checks in the model training pipeline so that biased models are not deployed.

Part B — Operationalizing Ethics in Deployment


Task 5 — Logging & Accountability


In [6]:
import datetime

prediction_log = []
MODEL_VERSION = "1.0.0"

def predict_and_log(applicant_features):
    pred = model.predict(applicant_features)[0]
    log_entry = {
        "timestamp": datetime.datetime.now().isoformat(),
        "model_version": MODEL_VERSION,
        "input_data": applicant_features.to_dict(orient='records')[0],
        "prediction": int(pred)
    }
    prediction_log.append(log_entry)
    return pred

# Example prediction logging
sample_applicant = X_test.iloc[[0]]
predict_and_log(sample_applicant)
prediction_log[:2]

[{'timestamp': '2025-08-13T11:09:28.951026',
  'model_version': '1.0.0',
  'input_data': {'Age': 44,
   'Years_Experience': 1,
   'Skill_Score': 44,
   'Gender_Male': False,
   'Gender_Other': False,
   'Ethnicity_Group_B': False,
   'Ethnicity_Group_C': False,
   'Education_High School': True,
   'Education_Masters': False,
   'Education_PhD': False},
  'prediction': 1}]

Task 6 — Real-Time Monitoring


In [7]:
# Simulated post-deployment monitoring
dp_gender = df_test.groupby("Gender")["y_pred"].mean().max() - df_test.groupby("Gender")["y_pred"].mean().min()
dp_ethnicity = df_test.groupby("Ethnicity")["y_pred"].mean().max() - df_test.groupby("Ethnicity")["y_pred"].mean().min()

print(f"Gender Disparity: {dp_gender:.3f}")
print(f"Ethnicity Disparity: {dp_ethnicity:.3f}")

if dp_gender > 0.1 or dp_ethnicity > 0.1:
    print("⚠️ ALERT: Fairness threshold exceeded — Investigation required")
else:
    print("✅ Fairness levels within acceptable range")

Gender Disparity: 0.302
Ethnicity Disparity: 0.284
⚠️ ALERT: Fairness threshold exceeded — Investigation required


Task 7 — Privacy Controls


In [8]:
import hashlib

data["Applicant_ID"] = data["Applicant_ID"].apply(lambda x: hashlib.sha256(str(x).encode()).hexdigest())
data.head()

,Applicant_ID,Age,Gender,Ethnicity,Education,Years_Experience,Skill_Score,Shortlisted
0,6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...,22,Male,Group_B,Bachelors,12,87,0
1,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...,48,Other,Group_B,Bachelors,12,89,1
2,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...,54,Male,Group_B,Bachelors,19,45,0
3,4b227777d4dd1fc61c6f884f48641d02b4d121d3fd328c...,58,Male,Group_C,Masters,8,41,0
4,ef2d127de37b942baad06145e54b0c619a1f22327b2ebb...,37,Male,Group_C,Bachelors,16,48,0


Task 8 — Inclusivity Feedback Loop


In [9]:
# Simulating underrepresented feedback cases
feedback = pd.DataFrame({
    "Age": [29, 32],
    "Gender": ["Other", "Other"],
    "Ethnicity": ["Group_C", "Group_C"],
    "Education": ["Bachelors", "Masters"],
    "Years_Experience": [5, 7],
    "Skill_Score": [82, 76],
    "Shortlisted": [1, 1]
})

# Append for future retraining
data_updated = pd.concat([data, feedback], ignore_index=True)
print("Updated dataset size:", data_updated.shape)

Updated dataset size: (1202, 8)


# Task 9 — Ethical Incident Simulation

Bias Complaint Example Plan

Detection — Monitoring dashboard flags Ethnicity bias > 0.15.

Investigation — Review model logs & fairness metrics, audit training data composition.

Communication — Report findings to compliance team & stakeholders.

Remediation — Retrain with balanced data + remove sensitive feature weights.

Prevention — Add stricter CI/CD bias thresholds.
